In [34]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.preprocessing import LabelEncoder
#LabelEncoder is used to convert categorical data to numerical data
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import seaborn as sns
import kagglehub
# We may use Decision Tree or XGBoost for better results
# from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor

In [35]:
# So how does XGBoost work?
# It builds an ensemble of weak learners (usually decision trees) in a sequential manner.
# Each new tree is trained to correct the errors made by the previous trees.
# The final prediction is made by combining the predictions of all the trees, typically through a weighted sum.
# XGBoost is known for its speed and performance, making it a popular choice for many machine learning tasks.
# Say that, there is an information_gain function that helps to decide the best feature to split the data at each node of the tree.
# Information gain is a measure used in decision trees to determine which feature to split the data on at each node.
# It quantifies the reduction in uncertainty or entropy about the target variable after splitting the data based on a particular feature.
# The feature that provides the highest information gain is chosen for the split, as it helps to create more homogeneous subsets of data.
# Information gain is calculated as the difference between the entropy of the original dataset and the weighted sum of the entropies of the subsets created by the split.
# Entropy is a measure of uncertainty or impurity in the data, and it is calculated using the formula:
# Entropy(S) = - Σ (p(x) * log2(p(x)))
# where p(x) is the proportion of instances belonging to class x in the dataset S.
# The information gain for a feature A is then calculated as:
# Information Gain(S, A) = Entropy(S) - Σ (|Sv| / |S|) * Entropy(Sv)
# where Sv is the subset of S for which feature A has value v, and |Sv| is the number of instances in Sv.
# The feature with the highest information gain is selected for the split at that node in the decision
# tree, as it helps to create more informative and effective splits that lead to better classification or regression performance.
# Note: Information gain is commonly used in algorithms like ID3 and C4.5 for building decision trees.
# However, XGBoost uses a more advanced approach called gradient boosting, which focuses on minimizing a specific loss function rather than relying solely on information gain.
# In gradient boosting, the model is built in a sequential manner, where each new tree is trained to correct the errors made by the previous trees.
# The objective is to minimize a loss function, such as mean squared error for regression tasks or log loss for classification tasks.
# So, how did we do it? We used the information gain concept to guide the splitting of the data at each node of the trees.
# This helps to create more informative splits that lead to better predictions.
# However, the actual implementation of XGBoost involves more complex techniques, such as regularization, handling missing values, and parallel processing, to optimize the model's performance.
# Which is something that we don't learn in basic ML courses.
# So, in summary, while information gain is a useful concept for understanding how decision trees work, XGBoost goes beyond that by using gradient boosting to build a powerful ensemble of trees that can achieve high accuracy on various machine learning tasks.
# Now, let's build this from scratch so that it can give you the whole concept of how it works without using the library.
# Note: Implementing XGBoost from scratch is a complex task that requires a deep understanding of machine learning concepts and algorithms.
# The following code provides a simplified version of XGBoost for educational purposes only.



### Let's build a Decision Tree Without library

In [36]:
# Please note that this is a basic implementation for a Decision Tree Regressor.
# Let's begin.

class DecisionTreeNode:
    def __init__(self, feature = None, threshold = None, left = None, right = None, value = None):
        self.feature = feature # Index of the feature to split on, example: 0 for the first feature and we will decide it based on information gain
        # Like for an instance, it will split based on the feature value at index 0 whether it is greater than or less than the threshold value
        self.threshold = threshold # Value to split the feature on, but we will decide it based on information gain
        self.left = left # Left child node, in competitive programming, we usually use None for null values
        self.right = right # Right child node, in competitive programming, we usually use None for null values
        self.value = value # Value to return if the node is a leaf node
    # So in conclusion, in a Decision tree, there will be features, and if a feature that got counted with information gain, and if it is greater than threshold, it will go to the left child node, otherwise, it will go to the right child node.
    # If it is a leaf node, it will return the value.
class DecisionTreeRegressor:
    def __init__(self, min_samples_split = 2, max_depth = 100, n_feats = None):
        self.min_samples_split = min_samples_split # Minimum number of samples required to split a node, how do we know this? We can decide it based on the dataset size
        self.max_depth = max_depth # Maximum depth of the tree to prevent overfitting
        self.n_feats = n_feats # Number of features to consider when looking for the best split # If None, then consider all features
        self.root = None # Root node of the tree # Initially, it is None
    def fit(self, X, y):
        self.n_feats = X.shape[1] if not self.n_feats else min(self.n_feats, X.shape[1]) # n_feats will take the number of features in X if it is None, otherwise, it will take the minimum of n_feats and the number of features in X
        self.root = self._grow_tree(X, y) # Grow the tree starting from the root node
    def _grow_tree(self, X, y, depth = 0):
        n_samples, n_features = X.shape
        if n_samples >= self.min_samples_split and depth < self.max_depth:
            feat_idxs = np.random.choice(n_features, self.n_feats, replace = False) # Randomly select n_feats features to consider for the best split
            best_feat, best_thresh = self.best_criteria(X, y, feat_idxs) # Get the best feature and threshold to split on, we will learn about this function later
            if best_feat is not None and best_thresh is not None: # Ensure both are not None
                left_idxs, right_idxs = self._split(X[:, best_feat], best_thresh) # Split the data based on the best feature and threshold
                if len(left_idxs) > 0 and len(right_idxs) > 0: # Only split if both sides have samples
                    left = self._grow_tree(X[left_idxs, :], y[left_idxs], depth + 1) # Recursively grow the left subtree
                    right = self._grow_tree(X[right_idxs, :], y[right_idxs], depth + 1) # Recursively grow the right subtree
                    return DecisionTreeNode(best_feat, best_thresh, left, right) # Return a decision node
        leaf_value = self._most_common_value(y) # If we cannot split further, return the most common value in y
        return DecisionTreeNode(value = leaf_value) # Return a leaf node
    def best_criteria(self, X, y, feat_idxs):
        best_gain = -1 # Initialize the best gain to -1
        split_idx, split_thresh = None, None # Initialize the best feature index and threshold to None
        for feat_idx in feat_idxs: # Iterate over the selected feature indices
            X_column = X[:, feat_idx] # Get the column of the feature to split on
            thresholds = np.unique(X_column) # Get the unique values of the feature to use as thresholds
            for threshold in thresholds: # Iterate over the unique thresholds
                gain = self._information_gain(y, X_column, threshold) # Calculate the information gain for the split
                if gain > best_gain: # If the gain is better than the best gain found so far
                    best_gain = gain # Update the best gain
                    split_idx = feat_idx # Update the best feature index
                    split_thresh = threshold # Update the best threshold
        return split_idx, split_thresh # Return the best feature index and threshold
    def _information_gain(self, y, X_column, split_thresh):
        # Parent entropy
        parent_entropy = self._entropy(y) # Calculate the entropy of the parent node
        # Generate split
        left_idxs, right_idxs = self._split(X_column, split_thresh) # Split the data based on the threshold
        if len(left_idxs) == 0 or len(right_idxs) == 0: # If one of the splits is empty, return 0 gain
            return 0
        # Weighted average child entropy
        n = len(y) # Total number of samples
        n_l, n_r = len(left_idxs), len(right_idxs) # Number of samples in the left and right splits
        e_l, e_r = self._entropy(y[left_idxs]), self._entropy(y[right_idxs]) # Entropy of the left and right splits
        child_entropy = (n_l / n) * e_l + (n_r / n) * e_r # Weighted average of the child entropies
        # Information gain is difference in entropy before vs. after split
        ig = parent_entropy - child_entropy # Calculate the information gain
        return ig # Return the information gain
    def _split(self, X_column, split_thresh):
        left_idxs = np.argwhere(X_column <= split_thresh).flatten() # Get the indices of the samples that go to the left split
        right_idxs = np.argwhere(X_column > split_thresh).flatten() # Get the indices of the samples that go to the right split
        return left_idxs, right_idxs # Return the indices of the left and right splits
    def _entropy(self, y):
        hist = np.bincount(y) # Count the occurrences of each class in y
        ps = hist / len(y) # Calculate the probabilities of each class
        return -np.sum([p * np.log2(p) for p in ps if p > 0]) # Calculate and return the entropy
    
    # Entropy is a measure of uncertainty or impurity in the data, and it is calculated using the formula:
    # Entropy(S) = - Σ (p(x) * log2(p(x)))
    # where p(x) is the proportion of instances belonging to class x in the dataset S
    # The information gain for a feature A is then calculated as:
    # Information Gain(S, A) = Entropy(S) - Σ (|Sv| / |S|) * Entropy(Sv)
    # where Sv is the subset of S for which feature A has value v, and |Sv| is the number of instances in Sv.
    # The feature with the highest information gain is selected for the split at that node in the decision
    # tree, as it helps to create more informative and effective splits that lead to better classification or regression performance.
    # Note: Information gain is commonly used in algorithms like ID3 and C4.5 for building decision trees.


    def _most_common_value(self, y):
        if len(y) == 0:
            return None  # or you can choose a default value, e.g., 0
        counter = np.bincount(y) # Count the occurrences of each class in y
        most_common = np.argmax(counter) # Get the class with the highest count
        return most_common # Return the most common class
    # So, in conclusion, the _most_common_value function is used to determine the most frequently occurring class label in the target variable y. This is particularly useful when creating leaf nodes in a decision tree, where we want to assign the most common class label to the leaf node when further splitting is not possible or necessary.
    def predict(self, X):
        return np.array([self._traverse_tree(x, self.root) for x in X]) # Traverse the tree for each sample in X and return the predictions
    # The predict function takes a 2D array X as input, where each row represents a sample with multiple features. It iterates over each sample (row) in X and calls the _traverse_tree function to navigate through the decision tree starting from the root node. The _traverse_tree function returns the predicted value for each sample, and the predict function collects these predictions into a NumPy array and returns it.
    # This allows us to make predictions for multiple samples at once by leveraging the structure of the decision tree.
    # Note: The predict function assumes that the decision tree has already been trained and is ready for making predictions.
    def _traverse_tree(self, x, node):
        if node.value is not None: # If the node is a leaf node
            return node.value # Return the value of the leaf node
        if x[node.feature] <= node.threshold: # If the feature value is less than or equal to the threshold
            return self._traverse_tree(x, node.left) # Traverse the left subtree
        return self._traverse_tree(x, node.right) # Traverse the right subtree
# The _traverse_tree function is a recursive function that navigates through the decision tree to make a prediction for a single sample x. It starts at the given node and checks if the node is a leaf node (i.e., it has a value). If it is a leaf node, it returns the value of that node as the prediction.

# 🌳 Decision Tree Regressor — Step-by-Step Explanation

---

## 🧩 1. Introduction
A **Decision Tree Regressor** predicts a continuous target value by recursively splitting data based on features that minimize impurity (e.g., variance, entropy, or Gini).  

Each internal node represents a **decision rule** (feature and threshold),  
and each leaf node represents a **predicted value**.

---

## 🧱 2. Node Structure (`DecisionTreeNode` Class)

Each node in the tree contains:

| Attribute | Description |
|------------|-------------|
| `feature` | Index of the feature used for the split |
| `threshold` | The numerical value used to split the feature |
| `left` | Pointer to the left child node (samples ≤ threshold) |
| `right` | Pointer to the right child node (samples > threshold) |
| `value` | The predicted value (for leaf nodes) |

A node is a **leaf** if it has a `value` and no children.

---

## ⚙️ 3. Decision Tree Regressor (`DecisionTreeRegressor` Class)

### Key Parameters

| Parameter | Description |
|------------|-------------|
| `min_samples_split` | Minimum samples required to split a node (prevents overfitting) |
| `max_depth` | Maximum tree depth (controls complexity) |
| `n_feats` | Number of features to consider at each split (adds randomness) |
| `root` | Root node of the tree, built after training |

---

## 🚀 4. Training the Model (`fit` Method)

1. Determine the number of features.  
2. Begin recursive tree construction using `_grow_tree()`.  
3. Find the **best feature** and **threshold** at each node to maximize **information gain**.

---

## 🌿 5. Growing the Tree (`_grow_tree` Method)

1. Check stopping conditions:
   - Fewer than `min_samples_split` samples, or  
   - Current depth ≥ `max_depth`.

2. Randomly select a subset of features (`feat_idxs`).  
3. Find the **best split** using `best_criteria()`.  
4. If a valid split is found:
   - Split the data into left and right subsets.  
   - Recursively grow the left and right subtrees.

5. If no split improves information gain → create a **leaf node** using `_most_common_value(y)`.

---

## 🔍 6. Finding the Best Split (`best_criteria` Method)

Loop through:
- Each feature `feat_idx` in the selected subset.  
- Each unique value in that feature (potential thresholds).

For every `(feature, threshold)` pair:
- Compute **information gain** using `_information_gain()`.  
- Keep the pair that yields the **highest gain**.

---

## 💡 7. Information Gain (`_information_gain` Method)

**Goal:** Measure how much a split reduces impurity.

**Steps:**
1. Compute parent node entropy:

\[
Entropy(S) = - \sum p_i \log_2(p_i)
\]

2. Split data into left and right subsets.  
3. Compute weighted average entropy of children.  
4. Calculate Information Gain:

\[
IG = Entropy(parent) - \left( \frac{n_L}{n} Entropy(left) + \frac{n_R}{n} Entropy(right) \right)
\]

> 🔸 In regression trees, this could be replaced by **variance reduction** instead of entropy.

---

## ✂️ 8. Splitting Data (`_split` Method)

Divide the dataset into two groups based on a threshold:

## 📉 9. Entropy Calculation (`_entropy` Method)

The **entropy** of a dataset measures its impurity or uncertainty — how "mixed" the classes are.  

In a binary classification problem:
- If all samples belong to one class → entropy = 0 (pure)
- If samples are evenly split → entropy = 1 (maximally uncertain)

**Formula:**

\[
Entropy(S) = - \sum_{i=1}^{c} p_i \log_2(p_i)
\]

Where:
- \( c \) = number of unique classes  
- \( p_i \) = probability of class \( i \) in the dataset  

**Example:**
If a node has 3 samples of class 0 and 1 sample of class 1:

\[
p_0 = \frac{3}{4}, \quad p_1 = \frac{1}{4}
\]
\[
Entropy = - \left( \frac{3}{4}\log_2\frac{3}{4} + \frac{1}{4}\log_2\frac{1}{4} \right) = 0.811
\]

---

## 🧠 10. Leaf Node Value (`_most_common_value` Method)

When no further splitting improves the information gain,  
the node becomes a **leaf node**, and its value is the **most common class** (for classification)  
or the **average target value** (for regression).

**Implementation Example:**

```python
def _most_common_value(self, y):
    values, counts = np.unique(y, return_counts=True)
    return values[np.argmax(counts)]
```

### 🔮 11. Prediction (predict Method)

Once the tree is trained, we use it to predict outcomes by traversing from the root down to a leaf node for each sample.

```python 
def predict(self, X):
    return np.array([self._traverse_tree(x, self.root) for x in X])
```

Each sample is passed through the _traverse_tree() method until it reaches a leaf value.

### 🧭 12. Tree Traversal (_traverse_tree Method)

At each node: Check if it’s a leaf node — if yes, return its stored value.

Otherwise, compare the feature with the threshold and go left or right.
```python
def _traverse_tree(self, x, node):
    if node.value is not None:
        return node.value
    if x[node.feature] <= node.threshold:
        return self._traverse_tree(x, node.left)
    return self._traverse_tree(x, node.right)
```

In [37]:
# Now, let's test our Decision Tree Regressor with the Bank Marketing dataset.
path = kagglehub.dataset_download("henriqueyamahata/bank-marketing")
data = pd.read_csv(path + "/bank-additional-full.csv", sep=';')
data.head()
# print(path)

X = data.drop('y', axis=1) #Or, you can use X = data[Categorical_features]
y = data['y']
# Note: Make sure to preprocess the data (handle categorical variables, missing values, etc.) before using it for training.
# For simplicity, let's assume X and y are already preprocessed and ready for training.
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#ValueError: could not convert string to float: 'blue-collar'
# This error occurs because the StandardScaler is trying to convert string values (categorical data) to float, which is not possible.
# To fix this, we need to preprocess the categorical features in the dataset before applying StandardScaler.
# One common approach is to use one-hot encoding or label encoding to convert categorical variables into numerical format. 
# Say, you have One-Hot encoder. 
# from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder()
# X_encoded = ohe.fit_transform(X)
# X_train, X_test, y_train, y_test = train_test_split(X_encoded, y, test_size=0.2, random_state=42)
# Let's use Label Encoding for simplicity
label_encoders = {}
for column in X.select_dtypes(include=['object']).columns:
    le = LabelEncoder()
    X[column] = le.fit_transform(X[column])
    label_encoders[column] = le
#But, what is with the complicated label encoding here? 
# We are storing the label encoders for each categorical column in a dictionary called label_encoders. This way, we can easily access the encoder for a specific column later if needed (for example, when making predictions on new data).
# This is useful because we might want to convert the encoded values back to their original categorical format some point, or we might want to apply the same encoding to new data that we want to predict on.
# Now, we can safely split the data and apply StandardScaler.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [38]:
# Encode y_train and y_test to integers
if 'y' not in label_encoders:
	label_encoders['y'] = LabelEncoder()
	label_encoders['y'].fit(y)

y_train_enc = label_encoders['y'].transform(y_train)
y_test_enc = label_encoders['y'].transform(y_test)

model = DecisionTreeRegressor(max_depth=10)
model.fit(X_train, y_train_enc)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test_enc, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

#90% accuracy? Nice! But, can we do better?

Accuracy: 90.67%


In [ ]:
# Now, let's use the library-based Decision Tree Classifier
from sklearn.tree import DecisionTreeClassifier

model = DecisionTreeClassifier(max_depth=10)
model.fit(X_train, y_train_enc)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test_enc, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 90.65%


In [ ]:
# Now, let's use XGBoos for better results
from xgboost import XGBClassifier  
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss') # Suppress the warning about label encoding
model.fit(X_train, y_train_enc)
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test_enc, y_pred)
print(f"Accuracy: {accuracy*100:.2f}%")

Accuracy: 91.55%


c:\Users\vpthi\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning: [15:35:22] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.

  bst.update(dtrain, iteration=i, fobj=obj)
